# Greedy

- problem: jumlah minimal tim dalam constraint yg diberikan
- constraint: is activity compatible with some team and deadline?

## todo
- fix any such team

In [66]:
import pandas as pd
from pprint import pprint

In [87]:
"""
https://docs.google.com/spreadsheets/d/1KxWTopBOX0lhMYJ0ij3u7sYs1s871_Vk/edit
"""

class Greedy():
    a = 1
    data = None
    df = None
    teamWork = {}
    teamCount = 0
    maxDayPerTeam = 30*3
    selectTeam = 0
    alwaysCompat = False
    
    def __init__(self):
        pass

    def load(self, fname):
        self.df = pd.read_csv(fname)
        return self
    
    def loadSet(self, fname):
        self.load(fname)
        self.data = self.df
        return self
    
    def reset(self):
        self.teamWork = {}
        self.teamCount = 0
        return self

    def sortBy(self, col):
        self.data = self.df.sort_values(col)
        return self
    
    def setMaxDayPerTeam(self, maxd):
        self.maxDayPerTeam = maxd
        return self

    # df compatible data
    def setData(self, data):
        self.data = data
        return self
    
    def addWorkToTeam(self, key, data):
        self.teamWork[key]['works'].append(data)
        current = self.teamWork[key]['totalDay']
        self.teamWork[key]['totalDay'] = current + data['dur']
        return self
    
    def isCompatibleToAnyTeam(self, data):
        compat = False
        if self.alwaysCompat:
            return True
        
        for t in g.teamWork.keys():
            lastWork = g.teamWork[t]['works'][-1]

            if lastWork['finish'] < data['start']:
                self.selectTeam = t
                compat = True
                break

        return compat

    # df compatible data
    def addTeam(self, key, data):
        self.teamWork[key] = data
        return self

    def isFeasible(self, idx, data):
        """
        data = {
            "team": self.teamCount,
            "work": d,
        }
        """
        try:
            # print(self.maxDayPerTeam)
            # print(self.teamWork[idx]['totalDay'])
            c1 = not len(self.teamWork.keys()) < 1
            c3 = self.isCompatibleToAnyTeam(data['work'])
            c2 = self.maxDayPerTeam >= self.teamWork[self.selectTeam]['totalDay']
            

            # todo : if compatible with some team
        except:
            return False
        cond = c1 & c2 & c3
        return cond
    
    def doGreedy(self):
        res = {}
        # for i,d in enumerate(self.data):
        for i,d in self.data.iterrows():
            fdata = {
                "team": self.teamCount,
                "work": d,
            }
            teamidx = str(self.teamCount)
            if self.isFeasible(teamidx, fdata):
                # add to existing teamwork
                if self.alwaysCompat:
                    self.selectTeam = teamidx
                
                self.addWorkToTeam(self.selectTeam, d)
            else:
                # add new data to teamwork
                idx = str(self.teamCount+1)
                self.addTeam(idx, {
                    "works":[],
                    "totalDay": 0,
                })
                self.addWorkToTeam(idx, d)
                self.teamCount += 1
#             print(self.teamWork)
        return res
    
    def showResult(self, prefix=""):
        """
        format: team, team totalDay, work counts
        """
        for t in g.teamWork.keys():
            cteamwork = g.teamWork[t]
            wt = cteamwork['works']
            ld = len(wt)
            msg = [t, str(cteamwork['totalDay']), f"{ld}"]
            print(prefix + ",".join(msg),",",["{} - {}".format(w['start'], w['finish']) for w in wt])

g = Greedy()
# fname = "const-greedy.csv"
fname = "https://gist.github.com/adib-enc/4408230ef5f91c00dae66406830348c0/raw/15cbdc1bbd67b54b07f1a9dcfd6727b1fc7cf43a/const-greedy.csv"
g.loadSet(fname)
g.data = g.data[0:6]
g.sortBy("start")

In [78]:
g.teamWork.keys()

dict_keys([])

In [79]:
g.data.describe()

,no,dur,dreward,reward total,cost,start,finish
count,18.000000,18.000000,18.0,18.000000,18.000000,18.000000,18.000000
mean,9.500000,24.222222,2000.0,48444.444444,5541.111111,95.333333,119.555556
std,5.338539,5.286560,0.0,10573.119203,8740.158761,55.728333,54.972958
min,1.000000,16.000000,2000.0,32000.000000,100.000000,1.000000,25.000000
25%,5.250000,20.000000,2000.0,40000.000000,1000.000000,52.750000,73.750000
50%,9.500000,24.000000,2000.0,48000.000000,1800.000000,106.000000,133.000000
75%,13.750000,28.750000,2000.0,57500.000000,3425.000000,138.250000,158.000000
max,18.000000,33.000000,2000.0,66000.000000,30000.000000,174.000000,194.000000


In [80]:
g.data

,no,dur,dreward,reward total,cost,start,finish
0,1,24,2000,48000,1200,1,25
12,13,24,2000,48000,1800,9,33
9,10,33,2000,66000,320,11,44
5,6,24,2000,48000,18000,41,65
13,14,18,2000,36000,2200,48,66
4,5,30,2000,60000,10000,67,97
7,8,24,2000,48000,120,78,102
17,18,18,2000,36000,2200,88,106
11,12,30,2000,60000,1000,101,131
16,17,24,2000,48000,1800,111,135


In [94]:
nscen = 1
header = [
    "scenario",
    "max_day",
    "feasible_criteria",
    "team",
    "team_total_day",
    "works_count",
    "detail",
]
print(",".join(header))
for days in [60, 90, 120, 180, 365]:
    for ty in ["dur", "dursf"]:
        # print(days, ty)
        g.setMaxDayPerTeam(days)
        if ty=="dur":
            g.alwaysCompat = True
        else:
            g.alwaysCompat = False
        
        result = g.reset().doGreedy()
        teamCount = len(g.teamWork.keys())
        
        # print(f"constraint: {g.maxDayPerTeam} hari per tim")
        # print(f"minimal tim: {teamCount}")
        prefix = ",".join([str(nscen), str(g.maxDayPerTeam), ty])
        g.showResult(prefix+",")
        nscen += 1
    # break

scenario,max_day,feasible_criteria,team,team_total_day,works_count,detail
1,60,dur,1,24,1 , ['1 - 25']
1,60,dur,2,81,3 , ['9 - 33', '11 - 44', '41 - 65']
1,60,dur,3,18,1 , ['48 - 66']
1,60,dur,4,30,1 , ['67 - 97']
1,60,dur,5,24,1 , ['78 - 102']
1,60,dur,6,18,1 , ['88 - 106']
1,60,dur,7,30,1 , ['101 - 131']
1,60,dur,8,24,1 , ['111 - 135']
1,60,dur,9,25,1 , ['117 - 142']
1,60,dur,10,30,1 , ['125 - 155']
1,60,dur,11,20,1 , ['130 - 150']
1,60,dur,12,18,1 , ['141 - 159']
1,60,dur,13,16,1 , ['153 - 169']
1,60,dur,14,33,1 , ['155 - 188']
1,60,dur,15,25,1 , ['166 - 191']
1,60,dur,16,20,1 , ['174 - 194']
2,60,dursf,1,78,3 , ['1 - 25', '41 - 65', '67 - 97']
2,60,dursf,2,66,3 , ['9 - 33', '48 - 66', '78 - 102']
2,60,dursf,3,51,2 , ['11 - 44', '88 - 106']
2,60,dursf,4,30,1 , ['101 - 131']
2,60,dursf,5,24,1 , ['111 - 135']
2,60,dursf,6,25,1 , ['117 - 142']
2,60,dursf,7,30,1 , ['125 - 155']
2,60,dursf,8,20,1 , ['130 - 150']
2,60,dursf,9,18,1 , ['141 - 159']
2,60,dursf,10,16,1 , ['153 - 169']
2,60,du

In [82]:
g.data.shape[0]

18

In [83]:
# https://www.toptal.com/developers/hastebin/ivecicixut.yaml
"""
function optimasi(){
    
    dur #jumlah hari yang diperlukan untuk pengerjaan
    cost #biaya yang diperlukan
    costcrash #biayapengahancuran 
    expdaycrash #lamanya waktu crash
    dailyreward #reward
    timeleft = 4*5*6  = 120 #waktu kerja selama 6 bulan
    task = #jumlah task

    #time
    #reward = 0
    #task = 0
    if(dur > 0 and timeleft > 0){
        timeleft = timeleft - dur;
        if(timeleft > 0 and expdaycrash > 0){
            timeleft = timeleft - expdaycrash;
            if(timeleft > 0){
                task = task + 1; #initiate task
            }
        }
    }

    #costreward
    if(cost != 0){
        totalcost = cost;
        if(costcrash != 0){
             totalcost = totalcost + costcrash
        }
        reward = dailyreward - totalcost;
    }

    print(task,reward,)
}
"""
# a
"fun"

'fun'